In [ ]:
import numpy as np
import cv2
import time

# define a video capture object 
vid = cv2.VideoCapture(0) 
time.sleep(3)

#high speed, low accuraccy 
#tracker = cv2.TrackerMOSSE_create()

#bit slower, more accurate
tracker = cv2.TrackerCSRT_create()

flag = 1

#starts loop
ret, frame = vid.read() 

bbox = cv2.selectROI("Tracking",frame, False)
tracker.init(frame,bbox)

def drawBox(frame,bbox):
    global flag 
    global x_prev, y_prev
    
    x, y, w, h = int(bbox[0]),int(bbox[1]),int(bbox[2]),int(bbox[3]) 
    cv2.rectangle(frame, (x,y), ((x+w),(y+h)), (255,0,255), 3,1)
    cv2.putText(frame,"Tracking",(75,75), cv2.FONT_HERSHEY_COMPLEX, 0.7,(0,255,0),2)

    #first frame, no movement
    if flag == 1:
        x_mov = 0
        y_mov = 0
        flag = 0
    else:
        x_mov = x-x_prev
        y_mov = y-y_prev
    
    x_prev = x
    y_prev = y
    return x_mov, y_mov

while(True): 
    timer = cv2.getTickCount() 
    # Capture the video frame 
    ret, frame = vid.read() 
    
    #searches for object in frame
    ret, bbox = tracker.update(frame)
    
    #if object is found in frame
    if ret:
        x_mov, y_mov = drawBox(frame,bbox)
        #difference in x and y movement in consecutive frames 
        print("x change: "+ str(x_mov))
        print("y change: "+ str(y_mov))
    #if object is not found in frame        
    else:
        cv2.putText(frame,"Lost",(75,75), cv2.FONT_HERSHEY_COMPLEX, 0.7,(0,0,255),2)
    
    fps = cv2.getTickFrequency()/(cv2.getTickCount()-timer)
    cv2.putText(frame,str(int(fps)),(75,50), cv2.FONT_HERSHEY_COMPLEX, 0.7,(0,0,255),2)
    
    # Display the resulting frame 
    cv2.imshow('frame', frame) 
    
    # press q to quit
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break
  

# After the loop release the cap object 
vid.release() 
# Destroy all the windows 
cv2.destroyAllWindows() 
